In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-78cdaf9d-0b94-cbaa-eab8-0bd36eb40c7b)


In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
from helper_functions import unzip_data, plot_loss_curves, compare_historys

--2024-05-25 14:43:59--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-05-25 14:43:59 (94.0 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data("nlp_getting_started.zip")

--2024-05-25 14:44:04--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.207, 142.251.2.207, 2607:f8b0:4023:c0b::cf, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2024-05-25 14:44:04 (155 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [4]:
import pandas as pd
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
shuffled_train_data = train_data.sample(frac=1,random_state=49)
shuffled_train_data.head()

,id,keyword,location,text,target
74,107,accident,North Carolina,I-77 Mile Marker 31 South Mooresville Iredell...,1
5387,7687,panic,Toronto,tomorrow's going to be a year since I went to ...,1
4259,6051,heat%20wave,"Arnhem, the Netherlands",Arnhem Weather - &lt;p&gt;An unrelenting and d...,1
2119,3045,death,Home of the Takers.,Y'all PUSSSSSSSSSY AND SHOOOK TO DEATH OF ME,0
6078,8684,sinkhole,"Haddonfield, NJ",Georgia sinkhole closes road swallows whole po...,1


In [6]:
shuffled_train_data.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [7]:
len(train_data) , len(test_data)

(7613, 3263)

In [8]:
import random
random_index = random.randint(0,len(train_data)-5)
for row in shuffled_train_data[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target = row
  print(f"Target : {target}","(Real Disaster)" if target > 0 else "(Not a Disaster)")
  print(f"Text: \n",{text})
  print("---------\n")

Target : 0 (Not a Disaster)
Text: 
 {'For those displaced by disasters label and legal status remain out of reach https://t.co/kJMgTEEklp'}
---------

Target : 0 (Not a Disaster)
Text: 
 {'Best believe all the wrong decisions are being made out here in these Memphis streets during this here rainstorm lol my folk doe'}
---------

Target : 1 (Real Disaster)
Text: 
 {'Officials: Alabama home quarantined over possible ... http://t.co/Rb2s0jmleJ'}
---------

Target : 1 (Real Disaster)
Text: 
 {'USGS EQ: M 1.2 - 23km S of Twentynine Palms California: Time2015-08-05 23:54:09 UTC2015-08-05 16:... http://t.co/T97JmbzOBO #EarthQuake'}
---------

Target : 1 (Real Disaster)
Text: 
 {'@binellithresa TY for the follow Go To http://t.co/UAN05TNkSW BRUTALLY ABUSED+DESOLATE&amp;LOST + HER LOVELY MUM DIES..Is it Murder?'}
---------



In [9]:
from sklearn.model_selection import train_test_split

train_text, val_text, train_labels, val_labels = train_test_split(shuffled_train_data["text"].to_numpy(),
                                                                  shuffled_train_data["target"].to_numpy(),
                                                                  test_size=0.1,
                                                                  random_state=49)

In [10]:
len(train_text) , len(train_labels) , len(val_text), len(val_labels)

(6851, 6851, 762, 762)

In [11]:
train_text[:5]

array(['Drunk twister is so hard ????',
       "Deepak Chopra's EPIC Twitter Meltdown http://t.co/ethgAGPy5G",
       '#NJTurnpike å_ #NJTurnpike Reopens Hours After Truck Fire In? http://t.co/oABJZtbVyZ http://t.co/GPBXRrDc07',
       "Hey @reddit - the concept of a 'quarantine' makes no sense if the people you've quarantined can just wander out &amp; about whenever they want",
       "'The Big Ten has their annual football media day but before we get into that here's some Cyclone hoops recruiting nuggets'"],
      dtype=object)

In [12]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode="int",
                                    output_sequence_length=None,
                                    pad_to_max_tokens=False)

In [13]:
round(sum([len(sentence.split()) for sentence in train_text]) / len(train_text))

15

In [14]:
max_words = 10000
max_length = 15
text_vector = TextVectorization(max_tokens=max_words,
                                output_mode='int',
                                output_sequence_length=max_length)

In [15]:
text_vector.adapt(train_text)

In [16]:
sample_text = "Deep Learning is one of the most important concept"
text_vector(sample_text)

<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([1150, 1655,    9,   57,    6,    2,  211, 5312, 3984,    0,    0,
          0,    0,    0,    0])>

In [17]:
# def Random_text_conversion():
#   random_index = random.randint(0,len(train_text))
#   print(f"Original Text : \n {train_text[random_index]} \n Converted Text : \n {text_vector(train_text[random_index])}" )

def Random_text_conversion():
  random_sentence = random.choice(train_text)
  print(f"Original Text : \n {random_sentence} \n Converted Text : \n {text_vector([random_sentence])}")

In [18]:
Random_text_conversion()

Original Text : 
 @Homukami Only URs and SRs matter Rs you'll be drowning in. Tho you're already drowning in Ns lol. 
 Converted Text : 
 [[   1  123 4309    7 8120 1508 3477 1240   21  438    4 1344  183  608
   438]]


In [19]:
from tensorflow.keras.layers import Embedding

emdedding_layer = Embedding(input_dim=max_words,
                            output_dim=128,
                            input_length=15)

In [20]:
def Random_text_emdeddings():
  random_sentence = random.choice(train_text)
  print(f"Original Text : \n {random_sentence} \n \n \n Converted Text : \n {emdedding_layer(text_vector([random_sentence]))}")

In [21]:
Random_text_emdeddings()

Original Text : 
 #AyekoRadio play Brasswork Agency - Crushed and Shaken http://t.co/Qh5axvhWH5 #Radio #NetLabel #ElectronicMusic #listen #CCMusic 
 
 
 Converted Text : 
 [[[-0.01848763  0.00516943  0.03300091 ... -0.00938242 -0.01038934
   -0.0280574 ]
  [-0.00156822 -0.04340839 -0.04486131 ...  0.03677474 -0.04115199
    0.04927856]
  [-0.01848763  0.00516943  0.03300091 ... -0.00938242 -0.01038934
   -0.0280574 ]
  ...
  [-0.01848763  0.00516943  0.03300091 ... -0.00938242 -0.01038934
   -0.0280574 ]
  [ 0.00980473  0.00336499 -0.0047061  ... -0.00804623 -0.02266735
   -0.00091206]
  [ 0.00980473  0.00336499 -0.0047061  ... -0.00804623 -0.02266735
   -0.00091206]]]


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tf-idf",TfidfVectorizer()),
    ("Naive-Bayes",MultinomialNB())
])
model_0.fit(train_text,train_labels)

Pipeline(steps=[('tf-idf', TfidfVectorizer()),
                ('Naive-Bayes', MultinomialNB())])

In [23]:
accuracy = model_0.score(val_text,val_labels)
accuracy

0.8097112860892388

In [24]:
train_text[:5],train_labels[:5]

(array(['Drunk twister is so hard ????',
        "Deepak Chopra's EPIC Twitter Meltdown http://t.co/ethgAGPy5G",
        '#NJTurnpike å_ #NJTurnpike Reopens Hours After Truck Fire In? http://t.co/oABJZtbVyZ http://t.co/GPBXRrDc07',
        "Hey @reddit - the concept of a 'quarantine' makes no sense if the people you've quarantined can just wander out &amp; about whenever they want",
        "'The Big Ten has their annual football media day but before we get into that here's some Cyclone hoops recruiting nuggets'"],
       dtype=object),
 array([0, 0, 1, 0, 1]))

In [25]:
model_0.predict(["Devastating earthquake in California. Numerous fatalities and injuries reported. Emergency services are working tirelessly on the scene. Our hearts go out to all affected. Please avoid the area and stay safe. #Earthquake #California #StaySafe"])

array([1])

In [26]:
y_pred = model_0.predict(val_text)
y_pred

array([1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1,

In [27]:
from sklearn.metrics import f1_score , accuracy_score, precision_score, recall_score, confusion_matrix
accuracy = accuracy_score(val_labels,y_pred)
precision = precision_score(val_labels,y_pred)
recall = recall_score(val_labels,y_pred)
f1 = f1_score(val_labels,y_pred)
confusion_matrix = confusion_matrix(val_labels,y_pred)
print(f"Acccuracy : {accuracy} \n Precision : {precision} \n Recall : {recall} \n F1-Score : {f1} \n Confusion Matrix : {confusion_matrix}")

Acccuracy : 0.8097112860892388 
 Precision : 0.8658008658008658 
 Recall : 0.6369426751592356 
 F1-Score : 0.7339449541284404 
 Confusion Matrix : [[417  31]
 [114 200]]


In [28]:
inputs = tf.keras.layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vector(inputs) # turn the input text into numbers
x = emdedding_layer(x) # create an embedding of the numerized numbers
x = tf.keras.layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")
model_1.compile(loss=tf.keras.losses.binary_crossentropy,optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])
model_1.fit(x=train_text,y=train_labels,epochs=5,validation_data=(val_text,val_labels))

Epoch 1/5
215/215 [==============================] - 10s 38ms/step - loss: 0.6140 - accuracy: 0.6871 - val_loss: 0.5243 - val_accuracy: 0.7651
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.4460 - accuracy: 0.8161 - val_loss: 0.4442 - val_accuracy: 0.8058
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.3517 - accuracy: 0.8586 - val_loss: 0.4252 - val_accuracy: 0.8110
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2882 - accuracy: 0.8899 - val_loss: 0.4318 - val_accuracy: 0.8084
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2406 - accuracy: 0.9117 - val_loss: 0.4452 - val_accuracy: 0.8058


In [29]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [30]:
model_1.evaluate(val_text,val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4452 - accuracy: 0.8058


[0.4452308714389801, 0.8057742714881897]

In [31]:
test_data.drop(["id","keyword","location"],axis=1,inplace=True)

In [32]:
test_data = test_data.to_numpy()
test_data

array([['Just happened a terrible car crash'],
       ['Heard about #earthquake is different cities, stay safe everyone.'],
       ['there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all'],
       ...,
       ['Green Line derailment in Chicago http://t.co/UtbXLcBIuY'],
       ['MEG issues Hazardous Weather Outlook (HWO) http://t.co/3X6RBQJHn3'],
       ['#CityofCalgary has activated its Municipal Emergency Plan. #yycstorm']],
      dtype=object)

In [33]:
test_preds = model_1.predict(test_data)
test_preds

102/102 [==============================] - 0s 2ms/step


array([[0.75351846],
       [0.5594343 ],
       [0.95646745],
       ...,
       [0.9437155 ],
       [0.8958776 ],
       [0.6916227 ]], dtype=float32)

In [34]:
def evaluate_model(labels,preds):
  accuracy = accuracy_score(labels,preds)
  precision = precision_score(labels,preds)
  recall = recall_score(labels,preds)
  f1 = f1_score(labels,preds)
  # confusion_matrix = confusion_matrix(labels,preds)
  print(f"Acccuracy : {accuracy} \n Precision : {precision} \n Recall : {recall} \n F1-Score : {f1}")

In [35]:
preds = model_1.predict([val_text])
evaluate_model(val_labels,tf.round(preds))

24/24 [==============================] - 0s 3ms/step
Acccuracy : 0.8057742782152231 
 Precision : 0.7922535211267606 
 Recall : 0.7165605095541401 
 F1-Score : 0.7525083612040135


In [36]:
inputs = tf.keras.layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vector(inputs) # turn the input text into numbers
x = emdedding_layer(x) # create an embedding of the numerized numbers
x = tf.keras.layers.GlobalMaxPooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")
model_1_1.compile(loss=tf.keras.losses.binary_crossentropy,optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])
model_1_1.fit(x=train_text,y=train_labels,epochs=5,validation_data=(val_text,val_labels))

Epoch 1/5
215/215 [==============================] - 10s 41ms/step - loss: 0.5042 - accuracy: 0.8059 - val_loss: 0.4726 - val_accuracy: 0.7940
Epoch 2/5
215/215 [==============================] - 2s 7ms/step - loss: 0.3185 - accuracy: 0.8889 - val_loss: 0.4588 - val_accuracy: 0.7940
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2492 - accuracy: 0.9072 - val_loss: 0.4739 - val_accuracy: 0.7940
Epoch 4/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2038 - accuracy: 0.9256 - val_loss: 0.4929 - val_accuracy: 0.7966
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1690 - accuracy: 0.9415 - val_loss: 0.5152 - val_accuracy: 0.7953


In [37]:
model_1_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [38]:
model_1_1.evaluate(val_text,val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.5152 - accuracy: 0.7953


[0.5152394771575928, 0.7952755689620972]

In [39]:
words_in_vocab = text_vector.get_vocabulary()
words_in_vocab

['',
 '[UNK]',
 'the',
 'a',
 'in',
 'to',
 'of',
 'and',
 'i',
 'is',
 'for',
 'on',
 'you',
 'my',
 'that',
 'it',
 'with',
 'at',
 'by',
 'this',
 'from',
 'be',
 'was',
 'have',
 'are',
 'like',
 'me',
 'as',
 'up',
 'just',
 'but',
 'so',
 'amp',
 'not',
 'im',
 'your',
 'will',
 'its',
 'out',
 'after',
 'an',
 'all',
 'no',
 'when',
 'fire',
 'has',
 'we',
 'if',
 'get',
 'about',
 'new',
 'now',
 'more',
 'via',
 'people',
 'or',
 'what',
 'one',
 'been',
 'he',
 'dont',
 'over',
 'they',
 'news',
 'who',
 'how',
 'into',
 'do',
 'were',
 'video',
 'us',
 'disaster',
 'can',
 'emergency',
 '2',
 'there',
 'than',
 'police',
 'would',
 'still',
 'his',
 'suicide',
 'some',
 'her',
 'back',
 'storm',
 'crash',
 'time',
 'body',
 'why',
 'got',
 'california',
 'burning',
 'them',
 'man',
 'off',
 'know',
 'rt',
 'had',
 'day',
 'see',
 'first',
 'world',
 'nuclear',
 'cant',
 'buildings',
 'our',
 'love',
 'fires',
 'their',
 'youtube',
 'going',
 'dead',
 'bomb',
 'attack',
 'two

In [40]:
model_1_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [41]:
embedding_weights = model_1_1.get_layer("embedding").get_weights()
embedding_weights[0].shape

(10000, 128)

In [42]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embedding_weights[0][index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [43]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [44]:
from tensorflow.keras import layers
input_layer = layers.Input(shape=(1,),dtype=tf.string)
x = text_vector(input_layer)
x = emdedding_layer(x)
x = layers.LSTM(64,return_sequences=True)(x)
x = layers.LSTM(64)(x)
x = layers.Dense(64,activation="relu")(x)
output_layer = layers.Dense(1,activation="sigmoid")(x)
model_2 = tf.keras.Model(input_layer,output_layer)

In [45]:
model_2.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 64)            49408     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                             

In [46]:
model_2.compile(loss=tf.keras.losses.binary_crossentropy,optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])
model_2_history = model_2.fit(x=train_text,y=train_labels,epochs=5,validation_data=(val_text,val_labels))

Epoch 1/5
215/215 [==============================] - 14s 44ms/step - loss: 0.2237 - accuracy: 0.9229 - val_loss: 0.5171 - val_accuracy: 0.7966
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.1509 - accuracy: 0.9451 - val_loss: 0.5143 - val_accuracy: 0.7913
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.1215 - accuracy: 0.9566 - val_loss: 0.7382 - val_accuracy: 0.7808
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0988 - accuracy: 0.9631 - val_loss: 0.8126 - val_accuracy: 0.7717
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0797 - accuracy: 0.9717 - val_loss: 1.1582 - val_accuracy: 0.7520


In [47]:
pred = model_2.predict(val_text)
tf.round(pred)

24/24 [==============================] - 1s 4ms/step


<tf.Tensor: shape=(762, 1), dtype=float32, numpy=
array([[1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
 

In [48]:
preds = tf.squeeze(tf.round(pred))

In [49]:
evaluate_model(val_labels,preds)

Acccuracy : 0.7519685039370079 
 Precision : 0.6888217522658611 
 Recall : 0.7261146496815286 
 F1-Score : 0.7069767441860466


In [56]:
## Using GRU
input_layer = tf.keras.Input(shape=(1,),dtype=tf.string)
x = text_vector(input_layer)
x = emdedding_layer(x)
x = tf.keras.layers.GRU(64)(x)
output_layer = tf.keras.layers.Dense(1,activation="sigmoid")(x)
model_3 = tf.keras.Model(input_layer,output_layer)

In [57]:
model_3.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru_2 (GRU)                 (None, 64)                37248     
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1317313 (5.03 MB)
Trainable params: 1317313 (5.03 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________

In [59]:
model_3.compile(loss=tf.keras.losses.binary_crossentropy,optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])
model_3_history = model_3.fit(x=train_text,y=train_labels,epochs=5,validation_data=(val_text,val_labels))

Epoch 1/5
215/215 [==============================] - 12s 39ms/step - loss: 0.0587 - accuracy: 0.9762 - val_loss: 1.1244 - val_accuracy: 0.7415
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0472 - accuracy: 0.9783 - val_loss: 1.0766 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0415 - accuracy: 0.9810 - val_loss: 1.2333 - val_accuracy: 0.7441
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0429 - accuracy: 0.9790 - val_loss: 1.3360 - val_accuracy: 0.7717
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0368 - accuracy: 0.9820 - val_loss: 1.4904 - val_accuracy: 0.7598


In [60]:
model_3_preds = model_3.predict(val_text)
model_3_preds = tf.squeeze(tf.round(model_3_preds))
model_3_preds

24/24 [==============================] - 1s 5ms/step


<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0.,
       1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0.,
       0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0.,
       0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0.,
       0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1.,
       0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0

In [61]:
evaluate_model(val_labels,model_3_preds)

Acccuracy : 0.7598425196850394 
 Precision : 0.7330960854092526 
 Recall : 0.6560509554140127 
 F1-Score : 0.692436974789916


In [64]:
##Bidirectional RNN
input_layers = tf.keras.layers.Input(shape=(1,),dtype=tf.string)
x = text_vector(input_layers)
x = emdedding_layer(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64,return_sequences=True))(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
output_layer = tf.keras.layers.Dense(1,activation="sigmoid")(x)
model_4 = tf.keras.Model(input_layers,output_layer)

In [65]:
model_4.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional_2 (Bidirecti  (None, 15, 128)           98816     
 onal)                                                           
                                                                 
 bidirectional_3 (Bidirecti  (None, 15, 128)           74496     
 onal)                                                           
                                                           

In [70]:
model_4.compile(loss=tf.keras.losses.binary_crossentropy,optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])
model_4_history = model_4.fit(x=train_text,y=train_labels,epochs=5,validation_data=(val_text,val_labels))

Epoch 1/5
215/215 [==============================] - 31s 113ms/step - loss: 0.0930 - accuracy: 0.9714 - val_loss: 1.1556 - val_accuracy: 0.7336
Epoch 2/5
215/215 [==============================] - 7s 32ms/step - loss: 0.0464 - accuracy: 0.9787 - val_loss: 1.3567 - val_accuracy: 0.7493
Epoch 3/5
215/215 [==============================] - 6s 28ms/step - loss: 0.0386 - accuracy: 0.9813 - val_loss: 1.3386 - val_accuracy: 0.7467
Epoch 4/5
215/215 [==============================] - 6s 30ms/step - loss: 0.0413 - accuracy: 0.9790 - val_loss: 1.5145 - val_accuracy: 0.7441
Epoch 5/5
215/215 [==============================] - 6s 30ms/step - loss: 0.0366 - accuracy: 0.9819 - val_loss: 1.4627 - val_accuracy: 0.7520


In [80]:
preds = model_4.predict(val_text)
preds

24/24 [==============================] - 2s 7ms/step


array([[9.99981165e-01],
       [9.99394298e-01],
       [9.45039392e-06],
       [4.75442335e-02],
       [9.98809934e-01],
       [9.25068676e-01],
       [1.27145776e-03],
       [1.12876602e-01],
       [3.03727955e-01],
       [9.99990106e-01],
       [9.99657393e-01],
       [7.09711967e-05],
       [9.99501467e-01],
       [3.59059253e-04],
       [9.99981165e-01],
       [9.99984264e-01],
       [3.01099881e-05],
       [9.99936819e-01],
       [1.31816778e-05],
       [9.99948978e-01],
       [3.65480125e-01],
       [1.83473695e-02],
       [9.16520548e-06],
       [5.67878829e-04],
       [4.51913802e-03],
       [2.19920790e-03],
       [2.63113052e-05],
       [9.99976516e-01],
       [1.29019551e-04],
       [9.99835610e-01],
       [9.99982357e-01],
       [9.99988556e-01],
       [9.99970555e-01],
       [9.96094167e-01],
       [9.99695897e-01],
       [7.61674892e-04],
       [9.99987245e-01],
       [3.84256840e-01],
       [3.12030222e-03],
       [9.99981523e-01],


In [81]:
preds = tf.squeeze(tf.round(preds))
preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0.,
       1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0.,
       0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0.,
       0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0.,
       0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0

In [83]:
evaluate_model(val_labels,preds)

Acccuracy : 0.7519685039370079 
 Precision : 0.7133105802047781 
 Recall : 0.6656050955414012 
 F1-Score : 0.6886326194398682
